In [0]:
import pandas as pd
import numpy as np
import pickle
import urllib.request, json
import datetime
import geopy.distance
from tqdm import tqdm

apikey = ''

In [0]:
lines_df = pd.read_hdf('public_transport_warsaw.h5', key='lines_df', mode='r')

In [0]:
print(lines_df.shape)
print(lines_df.columns)
print(lines_df.dtypes)

(345, 3)
Index(['line_nr', 'stops', 'stops_nr'], dtype='object')
line_nr     object
stops       object
stops_nr     int64
dtype: object


In [0]:
schedule_df = pd.read_hdf('public_transport_warsaw2.h5', key='schedule_df_2', mode='r')
stops_df = pd.read_hdf('public_transport_warsaw.h5', key='stops_df_4', mode='r')

In [0]:
print(stops_df.shape)
print(stops_df.columns)
print(stops_df.dtypes)
stops_df.head(1)

(7727, 19)
Index(['zespol', 'nazwa_zespolu', 'slupek', 'id_ulicy', 'kierunek', 'dlug_geo',
       'szer_geo', 'obowiazuje_od', 'line', 'id_nu', 'no_line', 'changed',
       'error', 'last_change', 'actual', 'deleted', 'dlug_f', 'szer_f',
       'coord'],
      dtype='object')
zespol                   object
nazwa_zespolu            object
slupek                   object
id_ulicy                 object
kierunek                 object
dlug_geo                 object
szer_geo                 object
obowiazuje_od    datetime64[ns]
line                     object
id_nu                    object
no_line                    bool
changed                   int64
error                      bool
last_change      datetime64[ns]
actual                     bool
deleted                    bool
dlug_f                  float64
szer_f                  float64
coord                    object
dtype: object


,zespol,nazwa_zespolu,slupek,id_ulicy,kierunek,dlug_geo,szer_geo,obowiazuje_od,line,id_nu,no_line,changed,error,last_change,actual,deleted,dlug_f,szer_f,coord
0,1001,Kijowska,01,2201,al.Zieleniecka,21.044827,52.248455,2019-09-19,"{138, 146, 509, 166, 147, 202, N71, N03, N02, ...",1001_01,False,1,False,2019-09-19,True,False,21.044827,52.248455,"(52.248455, 21.044826999999998)"


In [0]:
print(schedule_df.shape)
print(schedule_df.columns)
print(schedule_df.dtypes)
schedule_df.head(1)

(918719, 13)
Index(['line', 'id_nu', 'brygada', 'kierunek', 'trasa', 'czas', 'symbol_1',
       'symbol_2', 'idx', 'run_id', 'group_id', 'stop_count', 'stop_max'],
      dtype='object')
line          object
id_nu         object
brygada       object
kierunek      object
trasa         object
czas          object
symbol_1      object
symbol_2      object
idx            int64
run_id        object
group_id      object
stop_count     int64
stop_max       int64
dtype: object


,line,id_nu,brygada,kierunek,trasa,czas,symbol_1,symbol_2,idx,run_id,group_id,stop_count,stop_max
0,1,1006_03,12,Banacha,TP-BAN,04:50:00,null,null,0,1_12_0,1_12,50,50


In [0]:
stop_coords = stops_df[stops_df.actual].set_index('id_nu')[['coord']].to_dict('dict')['coord']

In [0]:
schedule_dict = {} # klucz (linia, brygada), zawiera listy z kolejnymi przystankami i planowanymi godzinami przyjazdu
                   # todo - kolejne pola mogą zawierać czas od początku trasy i odległość od początku trasy

for one in tqdm(schedule_df.itertuples()):       # 918719it [04:32, 3368.30it/s]
    mark = (one.line, one.brygada)
    if mark not in schedule_dict:
        stops = schedule_df[schedule_df.line.isin([one.line]) & 
                            schedule_df.brygada.isin([one.brygada])
                           ].sort_values(['czas'])[['id_nu', 'czas', 'kierunek', 'trasa']].values
        schedule_dict[mark] = stops
            
print(len(schedule_dict))

918719it [04:32, 3368.30it/s]


3599


In [0]:
from datetime import datetime
datetime.strptime(schedule_dict[('122', '4')][0][1], '%H:%M:%S')
schedule_dict[('122', '4')][0][1]

'05:41:00'

In [0]:
with open('schedule_dict.pickle', 'wb') as fp:
    pickle.dump(schedule_dict, fp)

In [0]:
with open('schedule_dict.pickle', 'rb') as fp:
    schedule_dict = pickle.load(fp)

In [0]:
schedule_dict[('122', '4')]

In [0]:
schedule_dict[('122', '4')][0]

['5050_08', '05:41:00', 'Gwiaździsta', 'TP-GWI', 0, 0.0, 0.0]

liczenie średniej prędkości dla całego dnia

In [0]:
235.60416 / (60120.0 / 60 / 60)
229.29598 / (59460.0 / 60 / 60)

13.88270312815338

In [0]:
def delta_seconds(start, stop):
    t_start = datetime.timedelta(hours=int(start[0:2]), minutes=int(start[3:5]), seconds=int(start[6:8]))
    t_stop = datetime.timedelta(hours=int(stop[0:2]), minutes=int(stop[3:5]), seconds=int(stop[6:8]))
    return (t_stop-t_start).total_seconds()

def distance(last_coord, new_coord):
    return round(geopy.distance.distance(last_coord, new_coord).kilometers, 5)

In [0]:
#dodawanie kolumn

for x in tqdm(schedule_dict.keys()):                     #3599/3599 [03:06<00:00, 19.33it/s]
    if len(x) > 4: continue    # second pass
    one = schedule_dict[x]
    t_first = one[0][1]
    p_prev = stop_coords[one[0][0]]
    direction = one[0][2]
    nr_round = 0
    dist_total = 0.0
    new_one = []
    for a in one:
        t_curr = a[1]
        p_curr = stop_coords[a[0]]
        dist_total += distance(p_prev, p_curr)
        p_prev = p_curr
        if a[2] != direction:
            direction = a[2]
            nr_round += 1
        b = [nr_round, delta_seconds(t_first, t_curr), round(dist_total,5)]
        new_one.append([*a, *b])
    schedule_dict[x] = new_one

100%|██████████████████████████████████████████████████████████████████████████████| 3599/3599 [03:06<00:00, 19.33it/s]


### schedule_df generator

In [0]:
def take_schedule(id_nu, line):
    [stop_id, stop_nr] = id_nu.split('_')
    api = 'https://api.um.warszawa.pl/api/action/dbtimetable_get?id=e923fa0e-d96c-43f9-ae6e-60518c9f3238' + \
        '&busstopId=' + stop_id + '&busstopNr=' + stop_nr + '&line=' + line + '&apikey=' + apikey
    try:
        with urllib.request.urlopen(api) as url:
            data_list = []
            data = json.loads(url.read().decode())['result']
            if (type(data) == str):
                raise Exception('API error')
            for one in data:
                row_dict = {}
                row_dict['line'] = line
                row_dict['id_nu'] = id_nu
                for col in one['values']:
                    row_dict[col['key']] = col['value']
                data_list.append(row_dict)
            return data_list
    except:
        print('error', stop_id, stop_nr, line)
        return

In [0]:
schedule_columns = ['line', 'id_nu', 'brygada', 'kierunek', 'trasa', 'czas', 'symbol_1', 'symbol_2']
if 'schedule_df' not in dir():
    schedule_df = pd.DataFrame([], columns=schedule_columns)
    print('starting new schedule_df')
else:
    print('adding to schedule_df')
    
for index, line in lines_df.iterrows():
    for stop in line['stops']:       
        print(line['line_nr'], stop)     
        one = take_schedule(stop, line['line_nr'])
        one_df = pd.DataFrame(one, columns=schedule_columns)
        schedule_df = pd.concat([schedule_df, one_df])

### schedule_df feature engineering

In [0]:
#schedule_df.to_hdf('public_transport_warsaw2.h5', key='schedule_df_2')

kolumna 'idx' jest do pominięcia - zły trop

In [0]:
schedule_df['idx'] = schedule_df.index
schedule_df['run_id'] = schedule_df[ ['line', 'brygada', 'idx'] ]\
            .apply(lambda x: (x['line']+'_'+x['brygada']+'_'+str(x['idx'])), axis=1)
schedule_df['group_id'] = schedule_df[ ['line', 'brygada'] ]\
            .apply(lambda x: (x['line']+'_'+x['brygada']), axis=1)
schedule_df['stop_count'] = schedule_df.groupby(['run_id']).transform(len) #slow
schedule_df['stop_max'] = schedule_df.groupby(['group_id'])['stop_count'].transform(max)
schedule_df['ride'] = schedule_df[ ['line', 'idx', 'kierunek'] ]\
            .apply(lambda x: (x['line']+'_'+str(x['idx'])+'_'+str(x['kierunek'])), axis=1)

In [0]:
schedule_df[schedule_df.line == '122'].trasa.value_counts()

TP-GOR      3168
TP-GWI      2800
TX-GWI02     238
TD-1GWI       27
Name: trasa, dtype: int64

In [0]:
schedule_df[schedule_df.line == '122'].kierunek.value_counts()

os.Górczewska    3168
Gwiaździsta      3065
Name: kierunek, dtype: int64

In [0]:
schedule_df[schedule_df.line.isin(['122'])
           & schedule_df.kierunek.isin(['Gwiaździsta'])
           ].brygada.value_counts().count()

14

In [0]:
schedule_df[schedule_df.line.isin(['122'])].idx.value_counts().value_counts()

71    80
70     6
72     1
61     1
Name: idx, dtype: int64

In [0]:
schedule_df[schedule_df.line.isin(['122'])].idx.value_counts().sort_index()

In [0]:
schedule_df[schedule_df.line.isin(['122'])
            & schedule_df.index.isin([1])
           ].sort_values(['kierunek', 'czas']).brygada.value_counts()

7      36
1      26
011     9
Name: brygada, dtype: int64

In [0]:
%%time 
schedule_df[schedule_df.line.isin(['122']) 
                  & schedule_df.kierunek.isin(['Gwiaździsta'])
                  & schedule_df.idx.isin(['10'])].sort_values(['czas'])['id_nu'].tolist()

In [0]:
stops_df[stops_df.id_nu.isin(['1006_03'])]

,zespol,nazwa_zespolu,slupek,id_ulicy,kierunek,dlug_geo,szer_geo,obowiazuje_od,line,id_nu,no_line,changed,error,last_change,actual,deleted,dlug_f,szer_f,coord
53,1006,rondo Starzyńskiego,03,2001,Wybrzeże Helskie,21.019762,52.263453,2019-09-19,"{1, 4, 6, 18, 28, 41}",1006_03,False,1,False,2019-09-19,True,False,21.019762,52.263453,"(52.263453000000005, 21.019762)"


In [0]:
schedule_df[schedule_df.id_nu.isin(['1006_03'])]['line'].value_counts()

1     211
18    102
6     102
28     98
41     95
4       6
Name: line, dtype: int64

In [0]:
schedule_df[schedule_df.id_nu.isin(['1006_03'])]['line']